In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import random
from zipfile import ZipFile
from sklearn.model_selection import train_test_split

In [ ]:
with ZipFile('pc6.zip') as zip_file:
    names = zip_file.namelist()
    with zip_file.open(names[0]) as data_file:
        df = pd.read_csv(data_file, index_col=0)

In [ ]:
pd.DataFrame(df.describe(include='all').transpose())

In [ ]:
variables = df.columns.drop('PC6')
data = df[variables]
correlation_matrix = data.corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5, mask=mask)
plt.title('Correlation Matrix')
plt.show()


In [ ]:
df['P_VROUW'] = df['VROUW']/df['INWONER']
df = df.drop(columns=['MAN', 'VROUW'])

In [ ]:
y = df['WOZWONING'] > df['WOZWONING'].mean()
X = df.drop(columns=['WOZWONING','PC6'])

Setting the random seeds for reproducibility

In [ ]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(torch.tensor(X.values, dtype=torch.float32), 
                                                    torch.tensor(y.values, dtype=torch.float32).reshape(-1, 1), 
                                                    train_size=0.7, 
                                                    shuffle=True,
                                                    random_state=0)
loader = torch.utils.data.DataLoader(list(zip(X_train, y_train)),
                                     batch_size=100,
                                     num_workers=4,
                                     worker_init_fn=seed_worker,
                                     generator=g)

In [ ]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.d1 = nn.Linear(26 * 26 * 32, 128)
        self.d2 = nn.Linear(128, 10)

    def forward(self, x):
        # 32x1x28x28 => 32x32x26x26
        x = self.conv1(x)
        x = F.relu(x)

        # flatten => 32 x (32*26*26)
        x = x.flatten(start_dim = 1)

        # 32 x (32*26*26) => 32x128
        x = self.d1(x)
        x = F.relu(x)

        # logits => 32x10
        logits = self.d2(x)
        out = F.softmax(logits, dim=1)
        return out